In [1]:
from skimage.io import imread
import numpy as np
import fnmatch
import os
import re
import random
import time

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import backend as K
from keras.backend.tensorflow_backend import set_session
from keras.utils import Sequence, to_categorical

import sys
if "E:\\git\\keras-resnet" not in sys.path:
    sys.path.append("E:\\git\\keras-resnet")
    print(sys.path)
    
import resnet


Using TensorFlow backend.


['', 'C:\\ProgramData\\Anaconda3\\python36.zip', 'C:\\ProgramData\\Anaconda3\\DLLs', 'C:\\ProgramData\\Anaconda3\\lib', 'C:\\ProgramData\\Anaconda3', 'C:\\Users\\hoang\\AppData\\Roaming\\Python\\Python36\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\ProgramData\\Anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\hoang\\.ipython', 'E:\\git\\keras-resnet']


In [2]:
#TODO check shuffle
class CustomDataGenerator(Sequence):

    def __init__(self, directory, batch_size, classes):
        self.directory = directory
        self.batch_size = batch_size
        self.classes = classes
        
        #create filenames, labels
        dirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]
        print(dirs)
        self.files = []
        for d in dirs:
            fullpath_dir = os.path.join(directory,d)
            new_elements = [(f,d) for f in os.listdir(fullpath_dir)]
            self.files.extend(new_elements)
        
        # shuffle
        random.shuffle(self.files)
        print(len(self.files))
    
    
    def getLabels(self, offsetStr, viewStr, altitudeStr): 
        offset = 0
        view = 0
        altitude = 0
        for i, e in enumerate(self.classes[0]):
            if e == offsetStr:
                offset = i
                break
            
        for i, e in enumerate(self.classes[1]):
            if e == viewStr:
                view = i
                break
        
        for i, e in enumerate(self.classes[2]):
            if e == altitudeStr:
                altitude = i
                break
                
        return offset, view, altitude
    
    
    def __len__(self):
        return int(np.ceil(len(self.files) / float(self.batch_size)))

    def on_epoch_end(self):
        'Shuffle for the next epoch'
        random.shuffle(self.files)
    
    def __getitem__(self, idx):
        batch_x = []
        batch_y1 = []
        batch_y2 = []
        batch_y3 = []
        batch = self.files[idx * self.batch_size : (idx + 1) * self.batch_size]
        #print(self.directory)
        #print ('files from getitem(): ', batch)
        p = re.compile('(left|center|right)Offset_(left|straight|right)View_(low|medium|high)Altitude')
        
        for (file, d) in batch:
            
            finds = p.findall(d)
            #TODO assert only finds has only 1 element [(_,_)]
            (offsetStr, viewStr, altitudeStr) =  finds[0]
            (offset, view, altitude) = self.getLabels(offsetStr, viewStr, altitudeStr)
            
            folder = os.path.join(self.directory, d)
            rgba = imread(os.path.join(folder, file))
            rgb = rgba[:,:,:3]
            
            input_array = rgb / 255.0
            
            batch_x.append(input_array)
            batch_y1.append(offset)
            batch_y2.append(view)
            batch_y3.append(altitude)
         
        return np.array(batch_x), [np.array(to_categorical(batch_y1, 3)), np.array(to_categorical(batch_y2, 3)), np.array(to_categorical(batch_y3, 3))]


In [3]:
#sanity test CustomDataGenerator
# directory = directory = "E:\\UAV_drone\\data\\training\\lateral_and_view_normal\\train"
# classes = [['left', 'center', 'right'], ['left', 'straight', 'right']]
# datagen = CustomDataGenerator(directory, 5, classes)
# datagen.__getitem__(0)


In [4]:
config = tf.ConfigProto()
#config.gpu_options.per_process_gpu_memory_fraction = 0.6
config.gpu_options.allow_growth = True  #dynamically grow the memory used on the GPU
set_session(tf.Session(config=config))

# dimensions of our images.
img_width, img_height = 512, 512

train_data_dir = 'E:\\UAV_drone\\data\\training\\constant_distance_to_lines\\train'
val_data_dir = 'E:\\UAV_drone\\data\\training\\constant_distance_to_lines\\val'
nb_train_samples = 118799
nb_validation_samples = 29802
epochs = 100
batch_size = 32
img_channels = 3
nb_classes = 3

# this is the augmentation configuration we will use for training
classes = [['left', 'center', 'right'], ['left', 'straight', 'right'], ['low', 'medium', 'high']]
train_datagen = CustomDataGenerator(train_data_dir, batch_size, classes)

# this is the augmentation configuration we will use for testing:
val_datagen = CustomDataGenerator(val_data_dir, batch_size, classes)


# checkpoint
current_directory = os.getcwd()
checkpoint_dir = os.path.join(current_directory, 'checkpoints')
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

file="model_resnet18_view-offset-altitude_{epoch:02d}.hdf5"
checkpoint =  ModelCheckpoint(os.path.join(checkpoint_dir, file), verbose=1)

NAME = "resnet18_view-offset-altitude_{}".format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME), write_graph = True, write_images=True)

['centerOffset_leftView_highAltitude', 'centerOffset_leftView_lowAltitude', 'centerOffset_leftView_mediumAltitude', 'centerOffset_rightView_highAltitude', 'centerOffset_rightView_lowAltitude', 'centerOffset_rightView_mediumAltitude', 'centerOffset_straightView_highAltitude', 'centerOffset_straightView_lowAltitude', 'centerOffset_straightView_mediumAltitude', 'leftOffset_leftView_mediumAltitude', 'leftOffset_rightView_mediumAltitude', 'leftOffset_straightView_highAltitude', 'leftOffset_straightView_lowAltitude', 'leftOffset_straightView_mediumAltitude', 'rightOffset_leftView_mediumAltitude', 'rightOffset_rightView_mediumAltitude', 'rightOffset_straightView_highAltitude', 'rightOffset_straightView_lowAltitude', 'rightOffset_straightView_mediumAltitude']
118646
['centerOffset_leftView_highAltitude', 'centerOffset_leftView_lowAltitude', 'centerOffset_leftView_mediumAltitude', 'centerOffset_rightView_highAltitude', 'centerOffset_rightView_lowAltitude', 'centerOffset_rightView_mediumAltitude

In [5]:
model = resnet.ResnetBuilder.build_resnet_18_uav((img_channels, img_width, img_height), nb_classes, multi_output=3)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 256, 256, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [6]:
model.fit_generator(
    train_datagen,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_datagen,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint, tensorboard])


Epoch 1/100
3712/3712 [==============================] - 6036s 2s/step - loss: 1.5368 - activation_18_loss: 0.4979 - activation_19_loss: 0.2986 - activation_20_loss: 0.4047 - activation_18_acc: 0.7759 - activation_19_acc: 0.8758 - activation_20_acc: 0.8150 - val_loss: 1.6723 - val_activation_18_loss: 0.5740 - val_activation_19_loss: 0.2191 - val_activation_20_loss: 0.6172 - val_activation_18_acc: 0.7553 - val_activation_19_acc: 0.9338 - val_activation_20_acc: 0.7428

Epoch 00001: saving model to C:\Users\hoang\WorkingSpace\TrainingModels\keras\checkpoints\model_resnet18_view-offset-altitude_01.hdf5
Epoch 2/100
3712/3712 [==============================] - 6014s 2s/step - loss: 0.6135 - activation_18_loss: 0.1869 - activation_19_loss: 0.0854 - activation_20_loss: 0.1074 - activation_18_acc: 0.9314 - activation_19_acc: 0.9738 - activation_20_acc: 0.9589 - val_loss: 1.3718 - val_activation_18_loss: 0.4492 - val_activation_19_loss: 0.3574 - val_activation_20_loss: 0.3497 - val_activation_18

KeyboardInterrupt: 

## continue training if necessary

In [ ]:
# from keras.models import load_model
# model_file = 'model-uav-view_offset_altitude-normal-newlossfunction-41.hdf5'
# model_full_path = os.path.join(checkpoint_dir, model_file)
# model = load_model(model_full_path)

# file="model-uav-view_offset_altitude-normal-newlossfunction-moredata-{epoch:02d}.hdf5"
# checkpoint =  ModelCheckpoint(os.path.join(checkpoint_dir, file), verbose=1)

# model.fit_generator(
#     train_datagen,
#     steps_per_epoch=nb_train_samples // batch_size,
#     epochs=epochs,
#     validation_data=val_datagen,
#     validation_steps=nb_validation_samples // batch_size,
#     callbacks=[checkpoint])